# Unity ML Agents
## Proximal Policy Optimization (PPO)
Contains an implementation of PPO as described [here](https://arxiv.org/abs/1707.06347).

In [1]:
import numpy as np
import os
import tensorflow as tf

from ppo.history import *
from ppo.models import *
from ppo.trainer import Trainer
from unityagents import *

Instructions for updating:
Use the retry module or similar alternatives.


### Hyperparameters

In [2]:
### General parameters
max_steps = 1e7 # Set maximum number of steps to run environment.
run_path = "deepHinge2" # The sub-directory name for model and summary statistics
load_model = False # Whether to load a saved model.
train_model = True # Whether to train the model.
summary_freq = 30000 # Frequency at which to save training statistics.
save_freq = 120000 # Frequency at which to save model.
env_name = "build/human" # Name of the training environment file.
curriculum_file = None

### Algorithm-specific parameters for tuning
gamma = 0.995 # Reward discount rate.
lambd = 0.900 # Lambda parameter for GAE.
time_horizon = 225 # How many steps to collect per agent before adding to buffer.
beta = 1e-4 # Strength of entropy regularization
num_epoch = 6 # Number of gradient descent steps per batch of experiences.
num_layers = 6 # Number of hidden layers between state/observation encoding and value/policy layers.
epsilon = 0.125 # Acceptable threshold around ratio of old and new policy probabilities. 0.1 - 0.3
buffer_size = 131072 # How large the experience buffer should be before gradient descent.
learning_rate = 5e-4 # Model learning rate. 1e-5 - 1e-3
hidden_units = 12 # Number of units in hidden layer.
batch_size = 4096 # How many experiences per gradient descent update step.
normalize = False
use_recurrent = True
sequence_length = 128 # 4 - 128
memory_size = 512 # 64 - 512

### Logging dictionary for hyperparameters
hyperparameter_dict = {'max_steps':max_steps, 'run_path':run_path, 'env_name':env_name,
    'curriculum_file':curriculum_file, 'gamma':gamma, 'lambd':lambd, 'time_horizon':time_horizon,
    'beta':beta, 'num_epoch':num_epoch, 'epsilon':epsilon, 'buffe_size':buffer_size,
    'leaning_rate':learning_rate, 'hidden_units':hidden_units, 'batch_size':batch_size,
    'use_recurrent':use_recurrent, 'sequence_length':sequence_length, 'memory_size':memory_size }

### Load the environment

In [3]:
env = UnityEnvironment(file_name=env_name, curriculum=curriculum_file)
print(str(env))
brain_name = env.external_brain_names[0]

INFO:unityagents:
'Academy' started successfully!


Unity Academy name: Academy
        Number of brains: 1
        Reset Parameters :
		
Unity brain name: brain
        Number of observations (per agent): 0
        State space type: continuous
        State space size (per agent): 88
        Action space type: continuous
        Action space size (per agent): 21
        Memory space size (per agent): 128
        Action descriptions: hode side, hode rist, hode nikk, rygg bukk, rygg len, l?r v l?p, l?r v strafe, legg v, fot v, l?r h l?p, l?r h strafe, legg h, fot h, skulder v klem, skulder v l?ft, arm v l?ft, arm v vri, skulder h klem, skulder h l?ft, arm h l?ft, arm h vri


### Train the Agent(s)

In [4]:
tf.reset_default_graph()

if curriculum_file == "None":
    curriculum_file = None


def get_progress():
    if curriculum_file is not None:
        if env._curriculum.measure_type == "progress":
            return steps / max_steps
        elif env._curriculum.measure_type == "reward":
            return last_reward
        else:
            return None
    else:
        return None

# Create the Tensorflow model graph
ppo_model = create_agent_model(env, lr=learning_rate,
                               h_size=hidden_units, epsilon=epsilon,
                               beta=beta, max_step=max_steps, 
                               normalize=normalize, num_layers=num_layers)

is_continuous = (env.brains[brain_name].action_space_type == "continuous")
use_observations = (env.brains[brain_name].number_observations > 0)
use_states = (env.brains[brain_name].state_space_size > 0)

model_path = './models/{}'.format(run_path)
summary_path = './summaries/{}'.format(run_path)

if not os.path.exists(model_path):
    os.makedirs(model_path)

if not os.path.exists(summary_path):
    os.makedirs(summary_path)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    # Instantiate model parameters
    if load_model:
        print('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(model_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        sess.run(init)
    steps, last_reward = sess.run([ppo_model.global_step, ppo_model.last_reward])    
    summary_writer = tf.summary.FileWriter(summary_path)
    info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
    trainer = Trainer(ppo_model, sess, info, is_continuous, use_observations, use_states, train_model)
    if train_model:
        trainer.write_text(summary_writer, 'Hyperparameters', hyperparameter_dict, steps)
    while steps <= max_steps:
        if env.global_done:
            info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
        # Decide and take an action
        new_info = trainer.take_action(info, env, brain_name, steps, normalize)
        info = new_info
        trainer.process_experiences(info, time_horizon, gamma, lambd)
        if len(trainer.training_buffer['actions']) > buffer_size and train_model:
            # Perform gradient descent with experience buffer
            trainer.update_model(batch_size, num_epoch)
        if steps % summary_freq == 0 and steps != 0 and train_model:
            # Write training statistics to tensorboard.
            trainer.write_summary(summary_writer, steps, env._curriculum.lesson_number)
        if steps % save_freq == 0 and steps != 0 and train_model:
            # Save Tensorflow model
            save_model(sess, model_path=model_path, steps=steps, saver=saver)
        steps += 1
        sess.run(ppo_model.increment_step)
        if len(trainer.stats['cumulative_reward']) > 0:
            mean_reward = np.mean(trainer.stats['cumulative_reward'])
            sess.run(ppo_model.update_reward, feed_dict={ppo_model.new_reward: mean_reward})
            last_reward = sess.run(ppo_model.last_reward)
    # Final save Tensorflow model
    if steps != 0 and train_model:
        save_model(sess, model_path=model_path, steps=steps, saver=saver)
env.close()
export_graph(model_path, env_name)

Step: 30000. Mean Reward: -15.98843579410669. Std of Reward: 28.910076963788324.
Step: 60000. Mean Reward: -18.276564261084587. Std of Reward: 25.536058257182916.
Step: 90000. Mean Reward: -20.82045520685747. Std of Reward: 27.69334831236108.
Step: 120000. Mean Reward: -17.137286117047864. Std of Reward: 25.57988079176553.
Saved Model
Step: 150000. Mean Reward: -2.9844845419585972. Std of Reward: 20.580420143417495.
Step: 180000. Mean Reward: 3.1037657254905167. Std of Reward: 13.078221101906687.
Step: 210000. Mean Reward: 3.184506629887369. Std of Reward: 13.325552552871363.
Step: 240000. Mean Reward: 3.314256928390164. Std of Reward: 13.945358419917767.
Saved Model
Step: 270000. Mean Reward: 6.586217877768073. Std of Reward: 17.17222862327832.
Step: 300000. Mean Reward: 27.63780128813141. Std of Reward: 10.534391445041068.
Step: 330000. Mean Reward: 26.85874057607911. Std of Reward: 10.467609802508374.
Step: 360000. Mean Reward: 27.68143120227237. Std of Reward: 11.383723890429957.
S

KeyboardInterrupt: 

### Export the trained Tensorflow graph
Once the model has been trained and saved, we can export it as a .bytes file which Unity can embed.

In [5]:
env.close()

In [6]:
export_graph(model_path, env_name)

INFO:tensorflow:Restoring parameters from ./models/deepHinge2/model-9000000.cptk


INFO:tensorflow:Restoring parameters from ./models/deepHinge2/model-9000000.cptk


INFO:tensorflow:Froze 15 variables.


INFO:tensorflow:Froze 15 variables.


Converted 15 variables to const ops.
